In [64]:
import pandas as pd

In [65]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_example = pd.read_csv('./data/sample_submission.csv')

In [66]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


# Feature engineering

In [67]:
# Extract deck and side
df_train.Cabin = df_train.Cabin.fillna('none/none/none')
df_train[['Deck','Num','Side']] = df_train.Cabin.str.split('/', expand=True)
df_train.loc[df_train.Deck=='none',['Deck','Side']] = [None, None]

# Remove blanck spaces in Destination
df_train.Destination = df_train.Destination.str.replace(' ','_')

# Dummy variables
dummies = ['HomePlanet','Destination','Deck','Side']

# Fill missing values with mode
for col in dummies:
    df_train[col].fillna(df_train[col].mode()[0], inplace=True)

# Dummie variables from this columns
df_train = pd.get_dummies(df_train, columns=dummies)

# Convert CryoSleep to dummy
df_train.CryoSleep.fillna(df_train.CryoSleep.mode()[0], inplace=True)
df_train.CryoSleep = df_train.CryoSleep.astype(int)

# Fill missing values and scale Age
df_train.Age.fillna(round(df_train.Age.mean()), inplace=True)
df_train.Age = df_train.Age / df_train.Age.max()

# Classify spend variables
spend = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

for col in spend:
    mean_value = df_train[col].mean()
    df_train[col+'_class'] = 0
    df_train.loc[(df_train[col]>0)&(df_train[col]<mean_value),col+'_class']=0.5
    df_train.loc[(df_train[col]>=mean_value),col+'_class'] = 1